In [ ]:
import numpy as np
from numpy.linalg import inv
from math import log

# 9.30 Gradient and Newton methods

$$
\min f(x) = - \sum_{i=1}^m log(1-a_i^Tx) - \sum_{i=1}^n log(1-x_i^2)
$$


a) Use the gradient method to solve the problem, using reasonable choices for the back-
tracking parameters, and a stopping criterion of the form k∇f (x)k 2 ≤ η. Plot the
objective function and step length versus iteration number. (Once you have deter-
mined p ⋆ to high accuracy, you can also plot f − p ⋆ versus iteration.) Experiment
with the backtracking parameters α and β to see their effect on the total number of
iterations required. Carry these experiments out for several instances of the problem,
of different sizes.

(b) Repeat using Newton’s method, with stopping criterion based on the Newton decre-
ment λ 2 . Look for quadratic convergence. You do not have to use an efficient method
to compute the Newton step, as in exercise 9.27; you can use a general purpose dense
solver, although it is better to use one that is based on a Cholesky factorization.

$$\bar{a}=\sum_{i=1}^m a_i$$


$$\nabla f(x)= \frac{\bar{a}}{1+a^Tx} + \frac{1}{1-x} - \frac{1}{1+x}$$

$$\nabla^2 f(x)=
\begin{bmatrix}
\frac{\bar{a}a_ix_i}{(1+a^Tx)^2} + \frac{1}{(1+x_i)^2}  + \frac{1}{(1-x_i)^2} & \frac{\bar{a}a_ix_j}{(1+a^Tx)^2}  \\
\frac{\bar{a}a_ix_j}{(1+a^Tx)^2}  & \dots
\end{bmatrix}
$$

In [ ]:
A = np.random.rand(m, n)

def f(x):
    y = []
    for i in range(n):
        yi = -sum(log(1-A[i].dot(x))) - sum(log(1-x[i]**2))
        y.append(yi)
    return np.array(y)
    
def df(x):
    y = []
    for i in range(n):
        yi = -sum(log(1-A[i].dot(x))) - sum(log(1-x[i]**2))
        y.append(yi)
    return np.array(y)
    

In [ ]:
def gradient_descent(f, df, alpha, beta):
    eta=1e-5 # Stopping criterion
    
    while df(x) < eta:
        # 1. Determine a descent direction ∆x
        deltax = df(x)
        # 2. Line search: choose a step size t > 0
        while f(x + t*deltax) > f(x) + alpha*t*df(x)*deltax:
                t = beta*t
        # 3. Update: x := x + t∆x
        x = x + t*df(x)
        
    return x, f(x)
    

In [ ]:
def newton_method(f, df, ddf, alpha, beta):
    epsilon = 1e-5
    
    lambd = 1e10
    while lambd/2 < epsilon:
        # 1. Determine a descent direction ∆x
        deltax = −inverse(ddf(x))*df(x)
        lambd = -df(x)*deltax
        # 2. Line search: choose a step size t > 0
        while f(x + t*deltax) > f(x) + alpha*t*df(x)*deltax:
                t = beta*t
                
        # 3. Update: x := x + t∆x
        x = x + t*df(x)

# 10.15 Equality constrained entropy maximization.

$$
\min ~f(x) = \sum_{i=1}^{n} x_i log(x_i)
$$
$$
subject~to~Ax=b
$$

(a) Standard Newton method. You can use initial point x (0) = x̂.

(b) Infeasible start Newton method. You can use initial point x (0) = x̂ (to compare with
the standard Newton method), and also the initial point x (0) = 1.

(c) Dual Newton method, i.e., the standard Newton method applied to the dual problem.




For the sake of symplicity we will calculate the entropy in nats and convert it to bits afterwards as that should not impact the final result

$\nabla f(x) = 1+ log(x_i)$

$\nabla^2 f(x) = 1/x_i$

In [105]:
#https://www.cs.cmu.edu/~ggordon/10725-F12/scribes/10725_Lecture12.pdf


# inv = np.linalg.inv

def f(x):
    return sum([xi*log(xi) for xi in x])

def df(x):
    return np.array([1+log(xi) for xi in x])

def ddf(x):
    A = np.identity(len(x))
    for i in range(len(x)):
        A[i,i] = 1/x[i]
    return A


def eqc_newton(f, df, ddf, A, alpha, beta, x0):
    lambd = 1e10
    epsilon = 1e-7
    x = x0
    
    for i in range(100):
        #1. Compute the Newton step and decrement ∆x nt , λ(x).
        print("#######################################")
        upper = np.concatenate([ddf(x), A.T], axis=1)
        lower = np.concatenate([A, np.zeros((A.shape[0], A.shape[0]))], axis=1)
        
        matrix = np.concatenate([upper, lower], axis =0)
        vector = np.append(-df(x), np.zeros(A.shape[0]))
        
        result = inv(matrix)@vector
        
        delta_x = result[0:len(x)]

        lambd = np.sqrt(delta_x.T@ddf(x)@delta_x)
        print(lambd)
        
        #2. Stopping criterion. quit if λ 2 /2 ≤ ǫ.
        if lambd/2 < epsilon:
            break
        
        #3. Line search. Choose step size t by backtracking line search.
        t = 1.0
        
#         #3.1 Guarantee x in dom f(x)
#         while min(x+t*delta_x) <= 0:
#             t = beta*t
        
        
        #3.1 Line search x in dom f(x)
        while f(x + t*delta_x) > f(x) + alpha*t*df(x).T@delta_x:
                t = beta*t
        
        #4. Update. x := x + t∆x nt .
        x = x + t*delta_x
        
        
    return
        
        

        
        
#n = 100 and p = 30 by choosing A randomly (checking that it has full rank)
n = 100
p = 30
A = np.random.rand(p, n)
x0 = np.abs(np.random.rand(n))
x0 = x0/np.linalg.norm(x0)
b = A@x0
assert np.linalg.matrix_rank(A) == p
alpha = 0.2
beta = 0.5


eqc_newton(f, df, ddf, A, alpha, beta, x0) 

#######################################
1.4218793694222616
#######################################
0.5678699436053058
#######################################
0.2064459829913621
#######################################
0.04056265824167877
#######################################
0.0013656236841484773
#######################################
1.5890196074238183e-06
#######################################
2.3644503128889965e-12


In [127]:
def eqc_infeasible_newton(f, df, ddf, A, b, alpha, beta, x0, v0):
    lambd = 1e10
    epsilon = 1e-7
    x = x0
    v = v0
    
    for i in range(100):
        #1. Compute the Newton step and decrement ∆x nt , λ(x).
        print("#######################################")
        upper = np.concatenate([ddf(x), A.T], axis=1)
        lower = np.concatenate([A, np.zeros((A.shape[0], A.shape[0]))], axis=1)
        
        matrix = np.concatenate([upper, lower], axis =0)
        vector = -np.append(df(x), A@x-b)
        
        result = inv(matrix)@vector
        
        delta_x = result[0:len(x)]
        w = result[len(x):]
        delta_v = w - v

        
        
        
        def r(x, v):
            primal = df(x) + A.T@v,
            dual = A@x-b
            return np.append(primal, dual)
        
        
        #2. Stopping criterion
        print(f(x), np.linalg.norm(r(x, v)), sum(A@x - b))
        if sum(A@x - b) <= epsilon and np.linalg.norm(r(x, v)) <= epsilon:
            break
            
                

        
        #3. Line search. Choose step size t by backtracking line search.
        t = 1.0
        #3.1 Guarantee x in dom f(x)
        while min(x+t*delta_x) <= 0:
            t = beta*t
        
        
        while np.linalg.norm(r(x+t*delta_x, v+t*delta_v)) < (1-alpha)*t*np.linalg.norm(r(x, v)):   
            t = beta*t
        
        #4. Update. x := x + t∆x nt .
        x = x + t*delta_x
        v = v+ t*delta_v
        
        
    return
        
        

        
        
#n = 100 and p = 30 by choosing A randomly (checking that it has full rank)
n = 100
p = 30
A = np.random.rand(p, n)
x0 = np.abs(np.random.rand(n))
x0 = x0/np.linalg.norm(x0)
b = A@x0
x0 = np.ones(n)
v0 = np.zeros(p)
assert np.linalg.matrix_rank(A) == p
alpha = 0.2
beta = 0.5


eqc_infeasible_newton(f, df, ddf, A, b, alpha, beta, x0, v0) 

#######################################
0.0 251.24796218445334 1373.5478194640045
#######################################
-33.17854067680879 125.5719364627221 686.7739097320042
#######################################
-36.29064502078926 62.76454814537694 343.3869548660023
#######################################
-32.00661684630126 31.39127813104149 171.69347743300048
#######################################
-27.506687921928172 15.722255317861872 85.84673871650027
#######################################
-24.396729287149753 7.886867540961565 42.92336935825019
#######################################
-22.559945052564334 3.9587057867012945 21.461684679125128
#######################################
-21.557580383033788 1.9857978029046053 10.730842339562589
#######################################
-21.0331001227455 0.9950650118487634 5.365421169781289
#######################################
-20.76468952738411 0.4981671955260785 2.6827105848906507
#######################################
-20.6288937